In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.functional as F 
import torch.nn as nn 
import torch 

# All skl imports go here
from sklearn import tree   # Decision Trees
from sklearn import svm    # svm

import sklearn as skl

In [16]:
# Load all data
train_data = pd.read_csv("training_labse_reduced.csv")
test_data = pd.read_csv("test_labse_reduced.csv")
validation_data = pd.read_csv("validation_labse_reduced.csv")

# Separate X's and y's from each other
FEATURE_COLUMNS = [x for x in train_data if x.startswith("c_")]
LABEL_COLUMN = "label"

X_train = train_data[FEATURE_COLUMNS]
Y_train = train_data[LABEL_COLUMN]

X_test = test_data[FEATURE_COLUMNS]
Y_test = test_data[LABEL_COLUMN]

X_val = validation_data[FEATURE_COLUMNS]
Y_val = validation_data[LABEL_COLUMN]

# These are used to run cross validation
X_train_val = pd.concat([X_train, X_val]) 
Y_train_val = pd.concat([Y_train + Y_test])

# Decision Trees

In [7]:
decision_tree_model = tree.DecisionTreeClassifier(
    criterion='entropy', 
    splitter = 'best',
    max_depth = 5
)


In [ ]:
decision_tree_model.fit(X_train, Y_train)
tree.plot_tree(decision_tree_model)

# SVMs

# MLP

# RNN

# Encoder-Only Transformer

# Test

In [ ]:
model_to_test = decision_tree_model

In [ ]:
#TODO: Add stuff for running the test set on the model